In [33]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

In [34]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Krmiljenje hitrostnega skoka hitrosti avtonomnega podvodnega vozila (AUV)

Dinamiko hitrostnega skoka (longitudinalne) hitrosti $u$ avtonomnega podvodnega vozila (AUV) lahko popišemo z

$$
(m+m_a)\dot{u} = -D(u-u_{current})+X,
$$

kjer je $ m = 40 $ kg masa vozila, $ m_a = 25 $ kg dodana masa, $ D = 12 $ Ns / m koeficient upora, $ X $ pogonska sila, ki jo ustvarja propeler, in $ u_ {current} $ hitrost vodnega toka. Motor propelerja je krmiljen s strani krmilnika, ki uravnava pogonsko silo; prenosno funkcijo krmilnika motorja (od zahtevane sile $ X_a $ do dejanske silo $ X $) lahko aproksimiramo z enotskim ojačanjem sistem drugega reda z lastno frekvenco $ \ omega_n = 1.3 $ rad / s in dušenjem $ \ xi = 0.7 $. Hitrostni skok hitrosti ocenimo z uporabo navigacijskega filtra.

Cilj je zasnova regulatorja, ki izpolnjuje naslednje zahteve:
- čas ustalitve krajši od 5 s (dosežena vrednost izhoda naj se razlikuje od tiste v stacionarnem stanju za 5%).,
- prenihaj manjši od 20%,
- brez odstopka v stacionarnem stanju v odzvu na spremembo hitrosti v obliki koračne funkcije,
- maksimalna absolutna sila $X$ znaša 60 N,

ob upoštevanju $u_{current} = 0$.


Prenosna funkcija od $X_a$ do $X$ je 

$$
X(s) = \frac{\omega_n^2}{s^2+2\xi\omega_ns+\omega_n^2} X_a(s)
$$

Iz česar sledi, da je, ob upoštevanju $z=\begin{bmatrix} z_1 & z_2 \end{bmatrix}^T$, možna realizacija krmilnika v vodljivostni kanonični obliki

\begin{cases}
    \dot{z} = \begin{bmatrix} 0 & 1 \\ -\omega_n^2 & -2\xi\omega_n \end{bmatrix}z + \begin{bmatrix} 0 \\ 1 \end{bmatrix}X_a \\
    X = \begin{bmatrix} \omega_n^2 & 0 \end{bmatrix}z.
\end{cases}

Celotna dinamika sistema je zaporedna vezava dinamik, katerima dodamo novo spremenljivko stanj, da dosežemo ničen odstopek v stacionarnem stanju. Ob upoštevanju vektorja stanj $x=\begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix}^T = \begin{bmatrix} u & z_1 & z_2 & x_a \end{bmatrix}^T$ with $\dot{x_a}=u-y_d$ lahko zapišemo

\begin{cases}
    \dot{x} = \begin{bmatrix} -\frac{D}{m+m_a} & \frac{\omega_n^2}{m+m_a} & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & -\omega_n^2 & -2\xi\omega_n & 0 \\ 1 & 0 & 0 & 0 \end{bmatrix}x + \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1 & 0 \\ 0 & -1 \end{bmatrix}\begin{bmatrix}X_a \\ y_d \end{bmatrix}\\
    y = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}x.
\end{cases}

Vodljivostna matrika $\mathcal{C}$

In [35]:
A = numpy.matrix([[-12/65,1.3**2/65,0,0],[0,0,1,0],[0,-1.3**2,-2*1.3*0.7,0],[1,0,0,0]])
B = numpy.matrix('0 0; 0 0; 1 0; 0 -1')
C = numpy.matrix('1 0 0 0; 0 0 0 1')
D = numpy.matrix('0 0; 0 0')

Ctrb = control.ctrb(A,B)
display(Markdown(bmatrix(Ctrb)))
# print(numpy.linalg.matrix_rank(Ctrb))

\begin{bmatrix}
  0. & 0. & 0. & 0. & 0.026 & 0. & -0.05212\\
  0.\\
  0. & 0. & 1. & 0. & -1.82 & 0. & 1.6224\\
  0.\\
  1. & 0. & -1.82 & 0. & 1.6224 & 0. & 0.123032\\
  0.\\
  0. & -1. & 0. & 0. & 0. & 0. & 0.026\\
  0.\\
\end{bmatrix}

ima rang enak 4, kar pomeni, da je sistem vodljiv.

Spoznavnostna matrika $\mathcal{O}$

In [36]:
Obsv = control.obsv(A,C)
display(Markdown(bmatrix(Obsv)))
# print(numpy.linalg.matrix_rank(Obsv))

\begin{bmatrix}
  1. & 0. & 0. & 0.\\
  0. & 0. & 0. & 1.\\
  -0.18461538 & 0.026 & 0. & 0.\\
  1. & 0. & 0. & 0.\\
  0.03408284 & -0.0048 & 0.026 & 0.\\
  -0.18461538 & 0.026 & 0. & 0.\\
  -0.00629222 & -0.04305385 & -0.05212 & 0.\\
  0.03408284 & -0.0048 & 0.026 & 0.\\
\end{bmatrix}

ima rang enak 4, kar pomeni, da je sistem tudi spoznaven.


### Načrtovanje regulatorja
#### Načrtovanje krmilnika

Možna rešitev je, da zamenjamo lokaciji realnega pola in pola,ki pripada integratorju, pri čemer preostalih polov krmilnika ne prestavljamo. 

As a possible solution we can change the location of the real pole and that of the integrator, while leaving the poles of the motor controller where they are. Izbrani poli so tako $-0.9\pm0.92$, $-1.3$ in $-1.1$.

#### Načrtovanje spoznavalnika

Z namenom, da krmilnik doseže dober prehodni pojav povezan z napako ocene stanj v primeru začetne napake pri oceni stanj, razporedimo pole spoznavalnika blizu -10 rad/s.

### Kako upravljati s tem interaktivnim primerom?
- Poizkusi spremeniti krmilnik tako, da omejiš absolutno vrednost $X$ na interval $\pm30$ N in hkrati zadostiš danim zahtevam.

In [37]:
# Preparatory cell

X0 = numpy.matrix('0.0; 0.0; 0.0; 0.0')
K = numpy.matrix([0,0,0,0])
L = numpy.matrix([[0, 0],[0, 0],[0, 0],[0, 0]])

X0w = matrixWidget(4,1)
X0w.setM(X0)
Kw = matrixWidget(1,4)
Kw.setM(K)
Lw = matrixWidget(4,2)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig4c = matrixWidget(2,1)
eig1c.setM(numpy.matrix([-1.3])) 
eig2c.setM(numpy.matrix([[-0.9],[-0.92]]))
eig3c.setM(numpy.matrix([-1.1]))
eig4c.setM(numpy.matrix([[-1],[-1]]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig3o = matrixWidget(1,1)
eig4o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-11.])) 
eig2o.setM(numpy.matrix([[-10.],[0.]]))
eig3o.setM(numpy.matrix([-12.]))
eig4o.setM(numpy.matrix([[-10.],[0.]]))

In [38]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Nastavi K in L', 'Nastavi lastne vrednosti'],
    value= 'Nastavi lastne vrednosti',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['brez kompleksnih lastnih vrednosti', 'dve kompleksni lastni vrednosti', 'štiri kompleksne lastne vrednosti'],
    value= 'dve kompleksni lastni vrednosti',
    description='Kompleksne lastne vrednosti:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulzna funkcija', 'koračna funkcija', 'sinusoidna funkcija', 'kvadratni val'],
    value='koračna funkcija',
    description='Vhod:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=2,
    min=0,
    max=4,
    step=0.1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.001,
    max=10,
    step=0.001,
    description='Perioda: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

gain_w2 = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

simTime = widgets.FloatText(
    value=7,
    description='',
    disabled=False
)

In [39]:
# Support functions

def eigen_choice(sele):
    if sele == 'brez kompleksnih lastnih vrednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig3o.children[0].children[0].disabled = False
        eig4o.children[0].children[0].disabled = False
        eig4o.children[1].children[0].disabled = True
        eig = 0
    if sele == 'dve kompleksni lastni vrednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = True
        eig4c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = False
        eig3o.children[0].children[0].disabled = False
        eig4o.children[0].children[0].disabled = True
        eig4o.children[1].children[0].disabled = True
        eig = 2
    if sele == 'štiri kompleksne lastne vrednosti':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = True
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig3o.children[0].children[0].disabled = True
        eig4o.children[0].children[0].disabled = False
        eig4o.children[1].children[0].disabled = False
        eig = 4
    return eig

def method_choice(selm):
    if selm == 'Nastavi K in L':
        method = 1
        sele.disabled = True
    if selm == 'Nastavi lastne vrednosti':
        method = 2
        sele.disabled = False
    return method

In [40]:
numU = 1
import warnings
# In order to suppress the warning for the precision of control.place
warnings.filterwarnings("ignore")

def main_callback2(X0w, K, L, eig1c, eig2c, eig3c, eig4c, eig1o, eig2o, eig3o, eig4o, u, period, selm, sele, selu, simTime, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(A-B[:,0:numU]*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eig1c[0,0]==eig2c[0,0] or eig1c[0,0]==eig3c[0,0] or eig1c[0,0]==eig4c[0,0]:
            eig1c[0,0] -= 0.0001
        if eig2c[0,0]==eig3c[0,0] or eig2c[0,0]==eig4c[0,0]:
            eig2c[0,0] -= 0.0002
        if eig1o[0,0]==eig2o[0,0] or eig1o[0,0]==eig3o[0,0] or eig1o[0,0]==eig4o[0,0]:
            eig1o[0,0] -= 0.0001
        if eig2o[0,0]==eig3o[0,0] or eig2o[0,0]==eig4o[0,0]:
            eig2o[0,0] -= 0.0002
        if eige == 0:
            K = control.acker(A, B[:,0:numU], [eig1c[0,0], eig2c[0,0], eig3c[0,0], eig4c[0,0]])
            Kw.setM(K)
            L = control.place(A.T, C.T, [eig1o[0,0], eig2o[0,0], eig3o[0,0], eig4o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(A, B[:,0:numU], [eig3c[0,0],
                                               eig1c[0,0],
                                               numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                               numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.place(A.T, C.T, [eig3o[0,0],
                                         eig1o[0,0],
                                         numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
        if eige == 4:
            K = control.acker(A, B[:,0:numU], [numpy.complex(eig4c[0,0],eig4c[1,0]), 
                                               numpy.complex(eig4c[0,0],-eig4c[1,0]),
                                               numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                               numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.place(A.T, C.T, [numpy.complex(eig4o[0,0],eig4o[1,0]), 
                                         numpy.complex(eig4o[0,0],-eig4o[1,0]), 
                                         numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    
    sys = control.ss(A,B,numpy.vstack((C,numpy.zeros((B.shape[1],C.shape[1])))),numpy.vstack((D,numpy.eye(B.shape[1]))))
    sysC = control.ss(numpy.zeros((1,1)),
                      numpy.zeros((1,numpy.shape(A)[0])),
                      numpy.zeros((numpy.shape(B[:,0:numU])[1],1)),
                      -K)
    
    sysE = control.ss(A-L*C,
                      numpy.hstack((L,B-L*D)),
                      numpy.eye(numpy.shape(A)[0]),
                      numpy.zeros((A.shape[0],C.shape[0]+B.shape[1])))
    
    sys_append = control.append(sys, sysE, sysC, control.ss(A,B,numpy.eye(A.shape[0]),numpy.zeros((A.shape[0],B.shape[1]))))
    Q = []
    # y in ingresso a sysE
    for i in range(C.shape[0]):
        Q.append([B.shape[1]+i+1, i+1])
    # u in ingresso a sysE
    for i in range(B.shape[1]):
        Q.append([B.shape[1]+C.shape[0]+i+1, C.shape[0]+i+1])
    # u in ingresso a sys
    for i in range(B[:,0:numU].shape[1]):
        Q.append([i+1, C.shape[0]+B.shape[1]+A.shape[0]+i+1])
    # u in ingresso al sistema che ha come uscite gli stati reali
    for i in range(B.shape[1]):
        Q.append([2*B.shape[1]+C.shape[0]+A.shape[0]+i+1, C.shape[0]+i+1])
    # xe in ingresso a sysC
    for i in range(A.shape[0]):
        Q.append([2*B.shape[1]+C.shape[0]+i+1, C.shape[0]+B.shape[1]+i+1])
        
    inputv = [i+numU+1 for i in range(B[:,numU:].shape[1])]
    outputv = [i+1 for i in range(numpy.shape(sys_append.C)[0])]
    # in order to avoid singular value exception
    try:
        sys_CL = control.connect(sys_append, Q, inputv, outputv)
    except:
        sys_CL = control.connect(sys_append, Q, inputv, outputv)
    
    dcgain = control.dcgain(sys_CL[0,0])
    gain_w2.value = dcgain
    if dcgain != 0:
        u1 = u/1
    else:
        print('Vnaprejšnje ojačanje je bilo nastavljeno na 0 in je bilo spremenjeno na vrednost 1')
        u1 = u/1
    print('Statično ojačanje zaprtozančnega sistema (od reference do izhoda) je: %.5f' %dcgain)
    
    X0w1 = numpy.zeros((A.shape[0],1))
    for j in range(A.shape[0]):
        X0w1 = numpy.vstack((X0w1,X0w[j]))
    X0w1 = numpy.vstack((X0w1,numpy.zeros((A.shape[0],1))))
    if simTime != 0:
        T = numpy.linspace(0, simTime, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)
      
    if selu == 'impulzna funkcija': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        U1 = [0 for t in range(0,len(T))]
        U1[0] = u1
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    if selu == 'koračna funkcija':
        U = [u for t in range(0,len(T))]
        U1 = [u1 for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    if selu == 'sinusoidna funkcija':
        U = u*numpy.sin(2*numpy.pi/period*T)
        U1 = u1*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    if selu == 'kvadratni val':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U1 = u1*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys_CL[:,0],T,U1,X0w1)
    
    step_info_dict = control.step_info(sys_CL[0,0],SettlingTimeThreshold=0.05,T=T)
    print('Informacije o odzivu sistema: \n\tČas vzpona [s] =',step_info_dict['RiseTime'],'\n\tČas ustalitve (5%) [s] =',step_info_dict['SettlingTime'],'\n\tPernihaj [%]=',step_info_dict['Overshoot'])
    print('Maksimalna vrednost x_2 (delež od 60 N)=', max(abs(yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+1]))/(60/1.3**2)*100)
    
    fig = plt.figure(num='Simulacija 1', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Odziv sistema')
    plt.ylabel('Izhod')
    plt.plot(T,yout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$y$','Referenca'])
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Vhod')
    plt.ylabel('$u$')
    plt.plot(T,yout[C.shape[0]])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Odziv stanj')
    plt.ylabel('Stanja')
    plt.plot(T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+1],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+2],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+3],
             T,[60/1.3**2 for i in range(len(T))],'r--',
             T,[-60/1.3**2 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$','$x_{4}$','$+60/\omega_n^2$','$-60/\omega_n^2$'])
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Napaka ocene stanj')
    plt.ylabel('Napaka ocene stanj')
    plt.plot(T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]]-yout[C.shape[0]+B.shape[1]],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+1]-yout[C.shape[0]+B.shape[1]+1],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+2]-yout[C.shape[0]+B.shape[1]+2],
             T,yout[C.shape[0]+B.shape[1]+B[:,numU:].shape[1]+A.shape[0]+3]-yout[C.shape[0]+B.shape[1]+3])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$e_{1}$','$e_{2}$','$e_{3}$','$e_{4}$'])
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.VBox([widgets.Label('K:',border=3), Kw, 
                                                        widgets.Label('Lastne vrednosti:',border=3),
                                                        widgets.HBox([eig1c, 
                                                                      eig2c, 
                                                                      eig3c,
                                                                      eig4c])])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Lastne vrednosti:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          eig3o,
                                          eig4o,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w,
                                          widgets.Label(' ',border=3)]),widgets.HBox([
                                          widgets.VBox([widgets.Label('Inverzno ojačanje reference:',border=3),
                                                        widgets.Label('Simulacijski čas [s]:',border=3)]),
                                          widgets.VBox([gain_w2,simTime])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('Referenca [m/s]:',border=3),
                                          u, 
                                          period, 
                                          START])])
out2 = widgets.interactive_output(main_callback2, {'X0w':X0w, 'K':Kw, 'L':Lw,
                                                   'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig4c':eig4c, 
                                                   'eig1o':eig1o, 'eig2o':eig2o, 'eig3o':eig3o, 'eig4o':eig4o, 
                                                   'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '860px'
display(out2, alltogether2)

Output(layout=Layout(height='860px'))